<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [5]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.5
Pandas:  1.0.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [21]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())



FileNotFoundError: [Errno 2] File dat/bikeshare.csv does not exist: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [6]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv',header = 0)
bikes.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [6]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [9]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [24]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name = 'Data', usecols = [0, 1, 2, 3, 4])
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [11]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-07-28T10:31:12.167Z,26.0696,142.5526,10,4.7,mb,NaN,137,1.073,0.68,...,2020-07-28T11:05:11.040Z,"Bonin Islands, Japan region",earthquake,8.7,1.5,0.074,55,reviewed,us,us


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [27]:
##url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

ValueError: No tables found

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [13]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [16]:
#ANSWER
bikes.shape



(17379, 17)

What are the column names in `bikes`?

In [17]:
#ANSWER
bikes.columns


Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [18]:
#ANSWER
bikes.dtypes


instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [29]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


What is the (row) index for this DataFrame?

In [22]:
#ANSWER
bikes.index



RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [23]:
#ANSWER
bikes['season']

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [30]:
print(bikes['atemp'].head())
print(type(bikes['atemp']))
print(bikes[['atemp']].head())
print(type(bikes[['atemp']]))


0    0.2879
1    0.2727
2    0.2727
3    0.2879
4    0.2879
Name: atemp, dtype: float64
<class 'pandas.core.series.Series'>
    atemp
0  0.2879
1  0.2727
2  0.2727
3  0.2879
4  0.2879
<class 'pandas.core.frame.DataFrame'>


In [31]:
print(bikes.atemp[:4])

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64


In [24]:
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [27]:
#ANSWER
bikes[['atemp']].head(n=4)


,atemp
0,0.2879
1,0.2727
2,0.2727
3,0.2879


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [28]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [30]:
#ANSWER
bikes[0:1]


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [31]:
#ANSWER
bikes[1:2]


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40


In [43]:
bikes.iloc[:, 1]
help(bikes.loc)


Help on _LocIndexer in module pandas.core.indexing object:

class _LocIndexer(_LocationIndexer)
 |  Access a group of rows and columns by label(s) or a boolean array.
 |  
 |  ``.loc[]`` is primarily label based, but may also be used with a
 |  boolean array.
 |  
 |  Allowed inputs are:
 |  
 |  - A single label, e.g. ``5`` or ``'a'``, (note that ``5`` is
 |    interpreted as a *label* of the index, and **never** as an
 |    integer position along the index).
 |  - A list or array of labels, e.g. ``['a', 'b', 'c']``.
 |  - A slice object with labels, e.g. ``'a':'f'``.
 |  
 |    .. warning:: Note that contrary to usual python slices, **both** the
 |        start and the stop are included
 |  
 |  - A boolean array of the same length as the axis being sliced,
 |    e.g. ``[True, False, True]``.
 |  - A ``callable`` function with one argument (the calling Series or
 |    DataFrame) and that returns valid output for indexing (one of the above)
 |  
 |  See more at :ref:`Selection by Labe

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [32]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [37]:
#ANSWER
empty = bikes.isnull()
bikes[empty == True].count()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [44]:
bikes.isnull().sum()


instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [38]:
#ANSWER
bikes[bikes.isnull() == True].count()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [45]:

#ANSWER

def halver(value):
    if value == 0 :
      print("Unable to divide by zero")
    else :
      print("half of this number is: " + str(value/2))
    return 

halver(1001)

halver(999)

halver(0)

half of this number is: 500.5
half of this number is: 499.5
Unable to divide by zero


In [48]:
def divide(dividend, divisor) :
    if divisor == 0 :
        quotient = np.nan
    else:
        quotient = dividend / divisor
    return (quotient)

print(divide ( 3,0))
divide ( 3,2)

nan


1.5

Apply the Pandas `isna` function to the following data objects:

In [40]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [45]:
#ANSWER
print(pd.isna(x))
print(pd.isna(y))

False
True


In [48]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)
print(pd.isna(array))

[[ 1. nan  3.]
 [ 4.  5. nan]]
[[False  True False]
 [False False  True]]


In [0]:
#ANSWER


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [49]:
#ANSWER
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [17]:
#ANSWER
print(bikes[['instant','dteday','windspeed']])


       instant      dteday  windspeed
0            1  2011-01-01     0.0000
1            2  2011-01-01     0.0000
2            3  2011-01-01     0.0000
3            4  2011-01-01     0.0000
4            5  2011-01-01     0.0000
...        ...         ...        ...
17374    17375  2012-12-31     0.1642
17375    17376  2012-12-31     0.1642
17376    17377  2012-12-31     0.1642
17377    17378  2012-12-31     0.1343
17378    17379  2012-12-31     0.1343

[17379 rows x 3 columns]


Compute and print the overall minimum and maximum of the numeric data columns:

In [20]:
##bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
##bikes_min, bikes_max

bikes[['instant','dteday','windspeed']].min()
bikes[['instant','dteday','windspeed']].max()


instant           17379
dteday       2012-12-31
windspeed        0.8507
dtype: object

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [50]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [51]:
#ANSWER
bikes['atemp'].quantile((0.1,0.25,0.5,0.75,0.9))

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [0]:
type(bikes['atemp'])

In [52]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
4061,4062,2011-06-23,3,0,6,3,0,4,1,2,0.72,0.6970,0.74,0.1642,1,4,5
12909,12910,2012-06-27,3,1,6,2,0,3,1,1,0.56,0.5303,0.49,0.1343,2,12,14
16883,16884,2012-12-11,4,1,12,6,0,2,1,2,0.34,0.3030,0.71,0.2985,0,118,118
2460,2461,2011-04-17,2,0,4,10,0,0,0,1,0.42,0.4242,0.44,0.4478,91,120,211
8000,8001,2011-12-05,4,0,12,0,0,1,1,1,0.32,0.3333,0.87,0.0896,3,21,24


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [53]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [0]:
#ANSWER


Here is a random sample of `atemp_level`:

In [54]:
atemp_level.sample(5)          

1114        (0.25, 0.5]
17148    (-0.001, 0.25]
10335       (0.25, 0.5]
3519        (0.75, 1.0]
14575       (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [55]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

5368     warm
6349     warm
11257    mild
8869     cool
4597     warm
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [59]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,mild
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,mild
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,mild
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,mild
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,mild


In [62]:
bikes[(bikes.atemp_level == 'mild')&(bikes.season == 2)]


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
1807,1808,2011-03-21,2,0,3,0,0,1,1,3,0.34,0.303,0.66,0.3881,2,11,13,mild
1808,1809,2011-03-21,2,0,3,1,0,1,1,2,0.34,0.303,0.71,0.3881,1,6,7,mild
1809,1810,2011-03-21,2,0,3,2,0,1,1,2,0.34,0.303,0.71,0.3284,1,5,6,mild
1810,1811,2011-03-21,2,0,3,3,0,1,1,2,0.34,0.303,0.71,0.3284,0,1,1,mild
1811,1812,2011-03-21,2,0,3,5,0,1,1,1,0.32,0.303,0.81,0.2985,1,1,2,mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12674,12675,2012-06-17,2,1,6,7,0,0,0,1,0.52,0.500,0.72,0.0896,18,30,48,mild
12694,12695,2012-06-18,2,1,6,3,0,1,1,2,0.52,0.500,0.83,0.2239,1,3,4,mild
12696,12697,2012-06-18,2,1,6,5,0,1,1,3,0.52,0.500,0.83,0.1343,1,22,23,mild
12697,12698,2012-06-18,2,1,6,6,0,1,1,3,0.52,0.500,0.88,0.1343,0,37,37,mild


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [63]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [0]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

## Check Column Names

Check column names and clean.

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.



> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



